In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import seaborn as sns
from pylab import rcParams
from collections import defaultdict
from scipy.optimize import curve_fit
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

     | 481kB 1.3MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-zywzw495/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import time
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [30]:
!ls "/content/drive/My Drive/dev-set"

aesthetic_visual_features  dev-set_ground-truth.csv    HOG	    ORB
C3D			   dev-set_video-captions.txt  InceptionV3  README.txt
ColorHistogram		   HMP			       LBP


In [31]:
Ground_Truth = '/content/drive/My Drive/dev-set/dev-set_ground-truth.csv'
data = pd.read_csv(Ground_Truth)
data.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


In [32]:

!pip install pyprind

In [0]:
# load labels and captions
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df


# load the captions
from pathlib import Path
inception_path = Path('Inception')

# load the ground truth values
label_path = Path('./dev-set_ground-truth.csv')


In [41]:
# video captions
video_captions = list(open('/content/drive/My Drive/dev-set/dev-set_video-captions.txt', 'r'))
video_captions = [i.split('\t') for i in video_captions]
video_captions = [[a, b.strip()] for a, b in video_captions]
video_captions = pd.DataFrame(video_captions, columns=['video', 'caption'])
video_captions.head(10)

,video,caption
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot
5,video12.webm,passing-by-older-man-reading
6,video13.webm,christmas-village-and-national-christmas-tree
7,video16.webm,unhappy-friends-looking-for-something-interest...
8,video17.webm,hoover-dam-tourists-traffic
9,video18.webm,happy-girl-stretching-and-relaxing-on-the-station


In [42]:
ground_truth_captions = video_captions.merge(ground_truth, left_on='video', right_on='video')
ground_truth_captions.head(10)

,video,caption,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,blonde-woman-is-massaged-tilt-down,0.924,34,0.846,13
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot,0.923,33,0.667,12
2,video6.webm,khr-gangsters,0.863,33,0.700,10
3,video8.webm,medical-helicopter-hovers-at-airport,0.922,33,0.818,11
4,video10.webm,couple-relaxing-on-picnic-crane-shot,0.950,34,0.900,10
5,video12.webm,passing-by-older-man-reading,0.783,74,0.750,12
6,video13.webm,christmas-village-and-national-christmas-tree,0.861,40,0.700,10
7,video16.webm,unhappy-friends-looking-for-something-interest...,0.834,33,0.857,14
8,video17.webm,hoover-dam-tourists-traffic,0.777,43,1.000,11
9,video18.webm,happy-girl-stretching-and-relaxing-on-the-station,0.863,33,1.000,9


In [0]:
feature_cols = ['short-term_memorability', 'nb_short-term_annotations', 'long-term_memorability', 'nb_long-term_annotations']
X = ground_truth_captions[feature_cols] # Features
y = ground_truth_captions.caption # Target variable

In [0]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [0]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [52]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.0
